# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [2]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
spaceship.drop(columns = ["PassengerId", "Name"], inplace=True)
spaceship["Cabin"] = spaceship["Cabin"].str[:1]

In [5]:
num = spaceship.select_dtypes("number")
cat = spaceship.select_dtypes("object")

In [6]:
normalizer = MinMaxScaler()
normalizer.fit(num)
num_norm = normalizer.transform(num)
num_norm = pd.DataFrame(num_norm, columns = num.columns)

In [7]:
cat = pd.get_dummies(cat)

In [8]:
df = num.join(cat).join(spaceship["Transported"])
df.dropna(inplace=True)

**Perform Train Test Split**

In [9]:
features = df.drop(columns = ["Transported"])
target = df["Transported"]

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [11]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [12]:
bagging_reg.fit(X_train, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [13]:
pred = bagging_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.2781746496917078
RMSE 0.37451196742685633
R2 score 0.4387998042548291


- Evaluate your model

So far, our best model was Bagging with a R-Squared of 0.44.

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [89]:
grid = {"n_estimators": [50, 100, 200,500],
        "max_samples": [250, 500, 1000]}

- Run Grid Search

In [90]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor())

In [91]:
model = GridSearchCV(estimator = bagging_reg, param_grid = grid, cv=5)

In [92]:
model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor()),
             param_grid={'max_samples': [250, 500, 1000],
                         'n_estimators': [50, 100, 200, 500]})

In [93]:
model.best_params_

{'max_samples': 500, 'n_estimators': 500}

In [94]:
best_model = model.best_estimator_

- Evaluate your model

In [95]:
pred = best_model.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test, y_test))

MAE 0.283910553368329
RMSE 0.3726509174540701
R2 score 0.44436345345537487


There is a little improvement in the results of the model by setting the best parameters using grid search as we can see in the lower MAE and RMSE, and higher R-squared.

- Run Random Search

In [96]:
grid = {"n_estimators": [int(x) for x in np.linspace(start = 50, stop = 1000, num = 5)],
        "max_samples": [int(x) for x in np.linspace(start = 250, stop = 1000, num = 5)]}

In [97]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor())

In [98]:
model = GridSearchCV(estimator = bagging_reg, param_grid = grid, cv=5)

In [99]:
model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor()),
             param_grid={'max_samples': [250, 437, 625, 812, 1000],
                         'n_estimators': [50, 287, 525, 762, 1000]})

In [100]:
model.best_params_

{'max_samples': 812, 'n_estimators': 1000}

In [101]:
best_model = model.best_estimator_

In [102]:
pred = best_model.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test, y_test))

MAE 0.2778407219930842
RMSE 0.3720121624323049
R2 score 0.4462666365156881


There is a little improvement in the results of the model by setting the best parameters using random search as we can see in the lower MAE and RMSE, and higher R-squared.